In [2]:
# Extremely Randomized Trees
# 초기 세팅
import pandas as pd

master = pd.read_csv("../dataset/master_3.csv", encoding='utf-8')

FileNotFoundError: [Errno 2] No such file or directory: '../dataset/master_3.csv'

In [ ]:
master = master.drop(["intime", "outtime", "ed_los", "outcome_icu_transfer_12h"], axis='columns')

master_X = master.loc[:, master.columns!='outcome_critical']
master_Y = master['outcome_critical']

master_Y.replace([False, True],[0,1], inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(master_X, master_Y, test_size=0.33, random_state=321)

In [ ]:
# ERT 러닝
from sklearn.ensemble import ExtraTreesClassifier

ertTree = ExtraTreesClassifier(n_estimators=10, random_state=2)
ertTree.fit(X_train, Y_train)

In [ ]:
print("훈련 세트 정확도: {:.3f}".format(ertTree.score(X_train, Y_train)))
print("테스트 세트 정확도: {:.3f}".format(ertTree.score(X_test, Y_test)))

In [ ]:
y_pred = ertTree.predict(X_test)

In [ ]:
setting(y_pred)
get_clf_eval(Y_test, y_pred)

In [ ]:
import numpy as np

importance = ertTree.feature_importances_
std = np.std([tree.feature_importances_ for tree in ertTree.estimators_], axis=0)
indices = np.argsort(importance)[::-1]

print("Feature ranking:")
for f in range(15):
    print("{}. feature {} ({:.3f})".format(f+1, master_X.columns[indices][f], importance[indices[f]]))

In [ ]:
n_features = master_X.shape[1]
plt.subplots(figsize=(10, 15))
plt.barh(range(n_features), ertTree.feature_importances_, align='center')
plt.yticks(np.arange(n_features), master_X.columns)
plt.xlabel("특성 중요도")
plt.ylabel("특성")
plt.ylim(-1, n_features)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def setting(y_pred):
    for i in range(len(y_pred)):
        if y_pred[i]>=.5:       # setting threshold to .5
           y_pred[i]=1
        else:  
           y_pred[i]=0

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0: .4f}, 정밀도: {1: .4f}, 재현율: {2: .4f},\nF1: {3: .4f}, AUC: {4: .4f}'.format(accuracy, precision, recall, f1, roc_auc))